In [3]:
import pandas as pd
import numpy as np
import os
import json
import shutil
import cv2

In [4]:
folder_dataset = 'yolo_dataset'

In [5]:
for f in [folder_dataset, folder_dataset+'/images', folder_dataset+'/images/train2017', folder_dataset+'/images/val2017', folder_dataset+'/labels/train2017', folder_dataset+'/labels/val2017']:
    if not os.path.exists(f):
        os.makedirs(f)

In [4]:
for folder, name in zip(['val', 'train'], ['europe', 'world']):
    for type_name, id_category in zip(['','_dog'], [1,2]):
        dataset = pd.read_csv('dataset'+type_name+'_'+name+'.csv')
        for index, row in dataset.iterrows():
            img = cv2.imread(name+type_name+'/'+str(int(row.id))+".png") 
            height, width, _ = img.shape
            file = open(folder_dataset+'/labels/'+folder+'2017'+'/'+str(id_category)+'_'+str(int(row.id))+".txt",'w')
            detections = np.array(row.detections.split(' ')).astype(np.int32)
            for i in range(int(row.wolves)):
                bbox = detections[i*4:(i+1)*4]
                point0 = bbox[:2]
                point1 = bbox[2:]
                point = [(point0[0]+point1[0])/2, (point0[1]+point1[1])/2]
                dims = [abs(point0[0]-point1[0]), abs(point0[1]-point1[1])]
                data = [(id_category-1), point[0]/width, point[1]/height, dims[0]/width, dims[1]/height]
                if data[1]>1 or data[2]>1:
                    print(str(id_category)+'_'+str(int(row.id)), 'W',width, 'H', height, bbox, 'P',point, 'D',dims)
                    print(data)
                data = [str(x) for x in data]
                file.write(' '.join(data)+'\n')
            file.close()
            shutil.copy(name+type_name+'/'+str(int(row.id))+".png", folder_dataset+'/images/'+folder+'2017'+'/'+str(id_category)+'_'+str(int(row.id))+".png")